<a href="https://colab.research.google.com/github/bnarath/TF_Developer/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we are going to cover some of the most fundamental concepts of tensors using TF 2.0

Topics being covered
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Unsing @tf.function(a way to speed up regular python functions)
* Using GPUs/TPUs with tensorflow
* Exercises


Methods
- tf.constant(tensor, shape, dtype) - It is an EagerTensor
- tf.Variable(tensor-initial, shape, dtype)
  - Assign values to variables by .assign()
- tf.random.Generator.from_seed(100)
  - tf.random(shape, minval, maxval, dtype)
  - tf.normal(shape, dtype)

- tf.random.set_seed(10)
  - tf.random.shuffle(not_shuffled)


Information
- tensor.ndim (len(shape))
- tensor.shape
- tf.size(tensor) - # of all elements
- Indexing (same as numpy indexing)
- tensor.shape[-1]
- tensor.numpy()


Add extra dimension
- tensor[ :, :, tf.newaxis] (tf.newaxis is same as None in numpy)
- tensor[ ..., tf.newaxis]
- tf.reshape(tensor, (tensor.shape[0], tensor.shape[1], 1))
- tf.expand_dims(tensor, -1)


Manipulating Tensors (Tensor Operations)

- Broadcasting Rules
  - Lower dimensional tensor extends to get dimension same as the higher
  - In each dimension, lengths should either match between 2 tensors or extendible
 
 - Multiply
    - tf.multiply (element wise) (alias of tf.math.multiply)
    - A*b (element wise)

- Add
  - tf.add (alias of tf.math.add)

- Matrix multiplication
  - AxB[i, j] is the dot product of ith row of A and jth column of B
  - tf.matmul
  - A @ B
  - tf.tensordot(A, B, axes=[[1], [0]]) - Axes defines vectors considered in dot product

- Transpose
  - tf.transpose



## Introduction to Tensors

In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.9.2'

## Create tensors using tf.Constant

In [2]:
scalar = tf.constant(9, dtype='int16')
scalar.ndim

0

In [3]:
vector = tf.constant([1,2,3])
vector, vector.ndim

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>, 1)

In [4]:
matrix = tf.constant([[1.,2], [3,4]])
matrix, matrix.ndim

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>, 2)

In [5]:
nd_matrix=tf.constant(np.arange(1, 243, 3), shape=(3,3,3,3))
nd_matrix.ndim

4

# What is a Tensor

n domensional array of numbers 
- n=0, scalar
- n=1, vector
- n=2, metrix


## Create tensors using tf.Variable

The Variable() constructor requires an initial value for the variable, which can be a Tensor of any type and shape. This initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods.

In [6]:
x = tf.Variable([1, 2])
x

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>

In [7]:
x.assign([2,3])
x[1].assign(10)
x

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([ 2, 10], dtype=int32)>

# Create random tensors

In [8]:
random1=tf.random.Generator.from_seed(100)
random1.uniform(
    (10,),
    minval=0,
    maxval=10,
    dtype=tf.int32
)
random1.normal(
    (10,),
    dtype=tf.float16
)

<tf.Tensor: shape=(10,), dtype=float16, numpy=
array([-0.785 ,  0.0869, -0.3286,  0.4028, -0.9795, -0.2349, -0.462 ,
        2.408 , -0.6445,  0.6567], dtype=float16)>

In [9]:
random2=tf.random.Generator.from_seed(100)
random2.uniform(
    (10,),
    minval=0,
    maxval=10,
    dtype=tf.int32
)
random2.normal(
    (10,),
    dtype=tf.float16
)

<tf.Tensor: shape=(10,), dtype=float16, numpy=
array([-0.785 ,  0.0869, -0.3286,  0.4028, -0.9795, -0.2349, -0.462 ,
        2.408 , -0.6445,  0.6567], dtype=float16)>

# Shuffle the order of elements in a tensor

In [10]:
not_shuffled = tf.constant([[10,7],
                            [1,2],
                            [2,4]])

In [11]:
tf.random.set_seed(10)
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  4],
       [ 1,  2]], dtype=int32)>

In [12]:
tf.random.set_seed(10)
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  4],
       [ 1,  2]], dtype=int32)>

## ⚓ Exercise: Produce 5 random tensors and shuffle them

https://www.tensorflow.org/api_docs/python/tf/random/set_seed

In [72]:
tf.random.set_seed(100)
random_gen = tf.random.Generator.from_seed(10)
a = random_gen.normal(shape=(1,10), mean=0, stddev=1)
b = random_gen.uniform(shape=(1,10), minval=0, maxval=10)
c = tf.reshape(
    tf.random.stateless_binomial(
    shape=[10], seed=[1, 1], counts=[10]*10, probs=np.arange(0.1, 1.1, 0.1), output_dtype=tf.float32), (1,10))
d = random_gen.normal(shape=(1,10), mean=10, stddev=1)
e = random_gen.normal(shape=(1,10), mean=0, stddev=2)
unshuffled = tf.concat([a,b,c,d,e], 0)
unshuffled, tf.random.shuffle(unshuffled, seed=10)

(<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
 array([[-2.9604465e-01, -2.1134205e-01,  1.0630016e-02,  1.5165398e+00,
          2.7305737e-01, -2.9925638e-01, -3.6523250e-01,  6.1883307e-01,
         -1.0130816e+00,  2.8291714e-01],
        [ 4.0812969e-01,  1.1846912e+00,  8.4382048e+00,  7.6974926e+00,
          6.6823254e+00,  3.8150179e+00,  3.5973561e+00,  3.4344697e+00,
          8.9352741e+00,  5.9334860e+00],
        [ 3.0000000e+00,  2.0000000e+00,  1.0000000e+00,  6.0000000e+00,
          2.0000000e+00,  5.0000000e+00,  6.0000000e+00,  8.0000000e+00,
          8.0000000e+00,  1.0000000e+01],
        [ 8.8429270e+00,  1.0771253e+01,  1.0319363e+01,  1.0404154e+01,
          1.0327567e+01,  1.1497083e+01,  9.8136797e+00,  1.2315570e+01,
          1.0519485e+01,  9.1210108e+00],
        [-1.6040002e+00, -4.1129370e+00,  1.0339310e+00,  2.2310841e+00,
          2.2762547e+00,  4.2137284e+00,  1.8501983e+00, -4.0433061e-01,
          2.8731544e+00,  7.6397896e-02]], dtype=flo

# Other ways to create tensors

In [75]:
print(tf.ones((1,10)))
tf.zeros((1,10))

tf.Tensor([[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(1, 10), dtype=float32)


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

# Turning numpy arrays into tensors

In [85]:
np_A = np.arange(1, 25, dtype=np.int32)
A = tf.constant(np_A, shape=(2,3,4))
B = tf.constant(np_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

# Getting information from tensors

In [108]:
A = tf.zeros(shape=(2, 3, 4, 5))
A.ndim, A.shape, tf.size(A), A[-1, -1, -1], A.dtype

(4,
 TensorShape([2, 3, 4, 5]),
 <tf.Tensor: shape=(), dtype=int32, numpy=120>,
 <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 tf.float32)

# Indexing Tensors

- like lists or numpy

In [121]:
random_gen = tf.random.Generator.from_seed(10)
A = random_gen.uniform((2,3,4,5), minval=0, maxval=10, dtype=tf.int32)

In [133]:
#Get first 2 elements of eatch dimension
print(A, A[:2, :2, :2, :2])

#Get first element in each dimension except the final one
print(A[0, 0, 0, :])

#Get first element in each dimension except the second last one
print(A[:1, :1, :, :1])

tf.Tensor(
[[[[6 8 2 4 4]
   [4 1 7 0 9]
   [2 9 3 9 6]
   [6 4 2 5 7]]

  [[9 6 9 1 3]
   [1 0 4 7 2]
   [5 3 0 3 6]
   [4 7 1 3 6]]

  [[1 7 4 9 0]
   [5 1 8 6 9]
   [8 1 8 2 9]
   [9 6 6 9 6]]]


 [[[3 0 3 2 5]
   [9 2 3 6 5]
   [7 0 2 6 2]
   [5 6 1 2 3]]

  [[8 0 4 9 7]
   [0 2 2 3 4]
   [8 3 7 9 3]
   [7 1 5 7 3]]

  [[1 7 5 0 7]
   [2 0 1 1 8]
   [2 3 8 9 6]
   [3 9 9 4 9]]]], shape=(2, 3, 4, 5), dtype=int32) tf.Tensor(
[[[[6 8]
   [4 1]]

  [[9 6]
   [1 0]]]


 [[[3 0]
   [9 2]]

  [[8 0]
   [0 2]]]], shape=(2, 2, 2, 2), dtype=int32)
tf.Tensor([6 8 2 4 4], shape=(5,), dtype=int32)
tf.Tensor(
[[[[6]
   [4]
   [2]
   [6]]]], shape=(1, 1, 4, 1), dtype=int32)


# Add extra dimension to tensor

In [134]:
rank_2_tensor = random_gen.uniform((2,2), minval=0, maxval=10, dtype=tf.int32)

In [140]:
rank_2_tensor.shape

TensorShape([2, 2])

In [163]:
rank_2_tensor[ :, :, tf.newaxis]
rank_2_tensor[ ..., tf.newaxis]
tf.reshape(rank_2_tensor, (rank_2_tensor.shape[0], rank_2_tensor.shape[1], 1))
tf.expand_dims(rank_2_tensor, -1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[2],
        [8]],

       [[3],
        [1]]], dtype=int32)>

# Tensor Manipulations (Tensor operations)

In [172]:
A = random_gen.uniform((2,2), minval=0, maxval=10, dtype=tf.int32)
b = tf.ones((2, 1), dtype=tf.int32)
A, b

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[6, 4],
        [8, 8]], dtype=int32)>, <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[1],
        [1]], dtype=int32)>)

In [173]:
A+10, A+b, A*b

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[16, 14],
        [18, 18]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[7, 5],
        [9, 9]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[6, 4],
        [8, 8]], dtype=int32)>)

In [175]:
tf.multiply(A, b)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[6, 4],
       [8, 8]], dtype=int32)>

In [176]:
A * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[60, 40],
       [80, 80]], dtype=int32)>

# Matrix multiplication

AxB[i, j] is the dot product of ith row of A and jth column of B

In [190]:
A = tf.Variable([[1, 2, 5], [7, 2, 1], [3, 3, 3]], dtype=tf.int32)
B = tf.constant([[3, 5], [6, 7], [1, 8]], dtype=tf.int32)

In [234]:
tf.math.equal(tf.matmul(A, B), A @ B)
tf.equal(A@B, tf.tensordot(A, B, [[1], [0]]))

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>